In [26]:
import pandas as pd
def wraptable(thing):
    return "\\begin{table} \n" + thing + "\n\\caption{whatever} \n" + "\\end{table}\n"
    


In [27]:
df = pd.read_csv('Table1_maxcut.csv')

In [28]:
main_body = (df.round(2).to_latex(index=False))
print(wraptable(main_body))

\begin{table} 
\begin{tabular}{lrrrrrr}
\toprule
   dataset &  gen\_n &      GAT &     GCNN &      GIN &  GatedGCNN &   LiftMP \\
\midrule
ErdosRenyi &   50.0 &   525.92 &   500.94 &   498.82 &     526.78 &   529.00 \\
ErdosRenyi &  100.0 &  1979.45 &  1890.10 &  1889.96 &    1978.78 &  1995.05 \\
ErdosRenyi &  400.0 & 16317.69 & 15692.12 & 15818.42 &   16188.85 & 16379.34 \\
\bottomrule
\end{tabular}

\caption{whatever} 
\end{table}



/tmp/ipykernel_1435529/1701652534.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  main_body = (df.round(2).to_latex(index=False))


# Table 2

In [ ]:
df = pd.read_csv('Table2_maxcut.csv')

In [29]:
main_body = df.iloc[:,1:].round(2).to_latex(index=False)

print(wraptable(main_body))



\begin{table} 
\begin{tabular}{rrrrrr}
\toprule
 gen\_n &      GAT &     GCNN &      GIN &  GatedGCNN &   LiftMP \\
\midrule
  50.0 &   525.92 &   500.94 &   498.82 &     526.78 &   529.00 \\
 100.0 &  1979.45 &  1890.10 &  1889.96 &    1978.78 &  1995.05 \\
 400.0 & 16317.69 & 15692.12 & 15818.42 &   16188.85 & 16379.34 \\
\bottomrule
\end{tabular}

\caption{whatever} 
\end{table}



/tmp/ipykernel_1435529/916021661.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  main_body = df.iloc[:,1:].round(2).to_latex(index=False)


In [ ]:
# instructions: 
2. add midrules

In [ ]:
\begin{tabular}{lrrrrrr}
\toprule
           Type &  Nmin &  Nmax &   LiftMP &  gurobi\_2.0 &  gurobi\_4.0 &  gurobi\_8.0 \\
\midrule
 BarabasiAlbert &  50.0 &  50.0 &   351.49 &      352.12 &      352.12 &      352.12 \\
 BarabasiAlbert & 100.0 & 100.0 &   717.05 &      719.93 &      720.00 &      720.22 \\
 BarabasiAlbert & 400.0 & 400.0 &  2198.11 &     2208.79 &     2213.00 &     2213.08 \\
\midrule
     ErdosRenyi &  50.0 &  50.0 &   529.00 &      530.12 &      530.17 &      530.19 \\
     ErdosRenyi & 100.0 & 100.0 &  1995.05 &     2002.89 &     2002.92 &     2002.98 \\
     ErdosRenyi & 400.0 & 400.0 & 16379.34 &    16493.98 &    16495.24 &    16495.50 \\
\midrule
PowerlawCluster &  50.0 &  50.0 &   345.74 &      346.42 &      346.42 &      346.42 \\
PowerlawCluster & 100.0 & 100.0 &   709.39 &      712.63 &      712.72 &      712.95 \\
PowerlawCluster & 400.0 & 400.0 &  2159.90 &     2169.78 &     2173.82 &     2173.95 \\
\midrule
  WattsStrogatz &  50.0 &  50.0 &   198.28 &      198.74 &      198.74 &      198.74 \\
  WattsStrogatz & 100.0 & 100.0 &   389.77 &      392.07 &      392.07 &      392.07 \\
  WattsStrogatz & 400.0 & 400.0 &  1165.90 &     1178.01 &     1178.46 &     1179.81 \\
\bottomrule
\end{tabular}